# Final Project - Super Blocks

In [1]:
# Import
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import floor
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind

## Introduction and Background

### Many cities and neighbourhoods in the United States, such as Los Angeles have been designed for extensive car use. While these models might have worked in the past, this dynamic is beginning to change as the result of overpopulation, rideshares and automation among other factors. Urban neighborhoods are no longer just places of living for large amounts of people, but rather an intertwined mesh of commercial activity, transport and residences (3). This does not fit the originally intended model - it is not efficient, causes pollution, and prevents children from being able to play in the streets.

### In Barcelona(2), city planners realized this problem and implemented a system where they blocked off certain urban areas to vehicle traffic, opening them up for commercial and community use. The results were astounding - with an increase shown in community engagement along with a decrease in pollution. Economic benefits have also been shown for the local businesses in these "superblocks" (4,5). As a result, we have decided to create a model that can figure out the best blocks to block out for such activity, geospatially and temporally.

### We will be using the number of restaurants, coffee shops, bars and transit stations to categorize the 'walkability' of different blocks, building a model analagous to walkscore (1). Furthermore, we will be using the Yelp API to temporally analyze foot traffic based on numbers and timing of reviews and checkins.

##### Referencess:
  ######  1)https://www.walkscore.com/professional/research.php
   ###### 2)https://www.theguardian.com/cities/2016/may/17/superblocks-rescue-barcelona-spain-plan-give-streets-back-residents
   ###### 3)https://www.wired.com/2017/04/brilliant-simplicity-new-yorks-new-times-square/ 
   ###### 4)http://krqe.com/2017/03/06/city-councilor-wants-wider-sidewalks-to-help-businesses-impacted-by-art/
   ###### 5)http://www.nyc.gov/html/dot/downloads/pdf/dot-economic-benefits-of-sustainable-streets.pdf

## Data Description

'FY2014 Ridership_Trolley_Sept2013Booking.csv' has the information about the number of people getting on and off the trolley for each ride for Sept2013.

'FY2014 Ridership_Trolley_Sept2013_Stops.csv' has the information about the location and other information about the trolley locations in San Diego.

## Data Cleaning / Pre-processing

### Active Businesses (Restaurants, cafe's etc) 

In [2]:
#Importing the original file for active business data in SD

fname = "business.csv"
business_df = pd.read_csv(fname)

In [3]:
#Importing the original file for active business data in SD

fname1 = "stops.csv"
stops_df = pd.read_csv(fname1)

FileNotFoundError: File b'stops.csv' does not exist

In [ ]:
#Importing the original file for active business data in SD

fname2 = "parking.csv"
parking_df = pd.read_csv(fname2)

In [ ]:
#Removing unneccesary columns and renaming to layman terms, filtering all business into places that serve food/drinks

business_df = business_df[['doing_bus_as_name','zip','naics_description','lat','lon']]
business_df = business_df.loc[(business_df['naics_description'] == 'full-service restaurants') |
               (business_df['naics_description'] == 'cafeterias') | 
               (business_df['naics_description'] == 'food services & drinking places') |
               (business_df['naics_description'] == 'limited-service eating places') |
               (business_df['naics_description'] == 'limited-service restaurants')  |
               (business_df['naics_description'] == 'mobile food services') |
               (business_df['naics_description'] == 'drinking places (alcoholic beverages)') |
               (business_df['naics_description'] == 'snack & nonalcoholic beverage bars')]
business_df.rename(columns = {'doing_bus_as_name':'Business title','naics_description':'Type of Place'}, inplace=True)
business_df = business_df.reset_index(drop=True)

In [ ]:
#Adding the 5 columns for the different time brackets

business_df.insert(2,'AM early','Null')
business_df.insert(3,'AM peak','Null')
business_df.insert(4,'Mid-day','Null')
business_df.insert(5,'PM peak','Null')
business_df.insert(6,'PM late','Null')

In [ ]:
#generalising the different categories into 3 categories: Only Food, Food & Drinks, Only Drinks

business_df.loc[(business_df['Type of Place'] == 'mobile food services') |
                (business_df['Type of Place'] == 'cafeterias') |
                (business_df['Type of Place'] == 'snack & nonalcoholic beverage bars') |
                (business_df['Type of Place'] == 'limited-service eating places') |
                (business_df['Type of Place'] == 'limited-service restaurants') 
                , 'Type of Place'] = 'Only Food'

business_df.loc[(business_df['Type of Place'] == 'full-service restaurants') 
                , 'Type of Place'] = 'Food & Drinks'

business_df.loc[(business_df['Type of Place'] == 'drinking places (alcoholic beverages)') |
                (business_df['Type of Place'] == 'food services & drinking places') 
                , 'Type of Place'] = 'Only Drinks'

In [ ]:
#Assigning values to different time brackets by assuming foot traffic according to the type of place 

business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('3','25','20','18','5')
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('7','28','25','35','25')
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('27','6','3','27','30') 

In [ ]:
#Assigning a total score to all three categories (summ of scores of all time brackets)

business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                'Total Score'] = '71'
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                'Total Score'] = '120'
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                'Total Score'] = '93'

In [ ]:
business_df

In [ ]:
business_df.to_csv('/Users/Devyaanshu/Pr_007/restaurants_data.csv')

### Transit Stops Dataset
The following cells take the original dataset and clean it so that we get the cleaned data, NOT MEANT TO BE RUN since we already have the cleaned dataset

In [ ]:
#Code which we used to clean the original dataset which is way too big for Github 

trips = pd.read_csv('FY2014 Ridership_Trolley_Sept2013Booking.csv')

In [ ]:
# Removing unnecessary columns: all we need is stop_id, passengers getting off on the station 
# and time_arrival to get the time split for the number of people getting off at a station at a time_period

trips = trips[['STOP_ID', 'PASSENGERS_OFF', 'TIME_ACTUAL_ARRIVE']]
trips.columns = ['stop_id', 'count', 'time']
trips

In [ ]:
##### clean the data, removing any na rows we will also remove all rows that have value 0

trips.dropna(how='any')
trips = trips[trips['count'] != 0]
trips

In [ ]:
# Using stop_id we can connect with lat/long. We will be grouping by pole_id to create a table with 
# the following columns: (stop_id, latitude, longitude, count_am_early, count_am_peak, count_midday, 
# count_pm_early, count_pm_late, count_daily)

#### Getting location data for all the stops using another dataset
stop_locs = pd.read_csv('FY2014 Ridership_Trolley_Sept2013_Stops.csv')
stop_locs = stop_locs[['STOP_ID', 'LAT', 'LON']]
stop_locs

In [ ]:
stop_locs.columns = ['stop_id', 'latitude', 'longitude']

In [ ]:
stops_df = trips.merge(stop_locs, how='left')
stops_df = stops_df.dropna(how='any')
stops_df

In [ ]:
##### grouping by stop_id and aggregating over the count values

stop_counts = stops_df.groupby(['stop_id']).agg('count')

##### remove unnecessary columns

stop_counts = stop_counts['time'] 
stop_counts = stop_counts.to_frame()
stop_counts['stop_id'] = stop_counts.index
stop_counts = stop_counts.merge(stop_locs, how='left')
stop_counts.columns = ['total_count', 'stop_id', 'latitude', 'longitude']
stop_counts

In [ ]:
##### We need to now find the number of days in the transactions dataset
##### We will be using this in order to get the count of transactions PER DAY

stops_df['time'] = pd.to_datetime(stops_df['time'])
dates = stops_df['time']
am_early_d = {}
am_peak_d = {}
midday_d = {}
pm_peak_d = {}
pm_late_d = {}

##### The time ranges for which we have split the transit ridership data are:
AM_Early = 12AM-6AM
AM_Late = 6AM-9AM
..
..


In [ ]:
#### Classify the time slot based on the times during the day
def classify(x): 
    hour = x.time().hour
    if hour <=6:
        return 'am_early'
    elif hour <=9:
        return 'am_peak'
    elif hour <=14:
        return 'midday'
    elif hour <=19:
        return 'pm_peak'
    else:
        return 'pm_late'
stops_df['time_slot'] = stops_df['time'].apply(classify)
stops_df

In [ ]:
##### Merging the two datasets by key value and column name

def checkSeriesColumn(s, col):
    val = False
    for row in s.keys().to_series().str.contains(col): 
        if(row == True):
            val = True
    return val

In [ ]:
### Setting the values of the transactions for each time period during the day
def set_temporal_counts(p_id):
    v_counts = stops_df.loc[stops_df['stop_id'] == p_id]['time_slot'].value_counts(dropna=False)
    stop_counts.loc[stop_counts['stop_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
    stop_counts.loc[stop_counts['stop_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']

stop_counts['stop_id'].apply(set_temporal_counts)

In [ ]:
stop_counts.to_csv('Stop_Counts.csv')

In [ ]:
#cleaned dataset for the transit spots
stops_df

### Parking meters 

###### Importing data and initial cleaning

In [ ]:
#importing dataset
parking = pd.read_csv('treas_parking_payments_2017_datasd.csv')

In [ ]:
#removing unnecessary columns: all we need is pole_id, time_start and meter_expire
parking = parking.drop(['uuid'], axis=1)
parking = parking.drop(['trans_amt'], axis=1)
parking = parking.drop(['pay_method'], axis=1)
parking = parking.drop(['meter_type'], axis=1)
parking = parking.drop(['meter_expire'], axis=1)

In [ ]:
#clean the data, removing any na rows 
parking.dropna(how='any')

###### Connecting with other dataset (with lat/long pairs)

In [ ]:
# 2.) using pole_id we can connect with lat/long. We will be grouping by pole_id to create
#     a table with the following columns:
#     (pole_id, latitude, longitude, count_am_early, count_am_peak, 
#        count_midday, count_pm_early, count_pm_late, count_daily)
# NOTE: we will need to decide whether to use raw numbers or averages of counts per section per day

In [ ]:
park_loc = pd.read_csv('treas_parking_meters_loc_datasd.csv')

In [ ]:
park_loc = park_loc[['pole', 'longitude', 'latitude']]
park_loc.columns = ['pole_id', 'longitude', 'latitude']
park_loc = park_loc.dropna(how='any')

In [ ]:
park_df = parking.merge(park_loc, how='left')
park_df = park_df.dropna(how='any')
park_df

In [ ]:
park_counts = park_df.groupby(['pole_id']).agg('count')
park_counts = park_counts['trans_start'] #remove unnecessary columns
park_counts = park_counts.to_frame()
park_counts['pole_id'] = park_counts.index
park_counts = park_counts.merge(park_loc, how='left')
park_counts.columns = ['total_count', 'pole_id', 'longitude', 'latitude']
park_counts

###### A

In [ ]:
# we need to now find the number of days in the transactions dataset
# we will be using this in order to get the count of transactions PER DAY
park_df['trans_start'] = pd.to_datetime(park_df['trans_start'])
dates = park_df['trans_start']
am_early_d = {}
am_peak_d = {}
midday_d = {}
pm_peak_d = {}
pm_late_d = {}

def classify(x): 
    hour = x.time().hour
    if hour <=6:
        return 'am_early'
    elif hour <=9:
        return 'am_peak'
    elif hour <=14:
        return 'midday'
    elif hour <=19:
        return 'pm_peak'
    else:
        return 'pm_late'
park_df['time_slot'] = park_df['trans_start'].apply(classify)
park_df

In [ ]:
def checkSeriesColumn(s, col):
    val = False
    for row in s.keys().to_series().str.contains(col): 
        if(row == True):
            val = True
    return val
def set_temporal_counts(p_id):
    v_counts = park_df.loc[park_df['pole_id'] == p_id]['time_slot'].value_counts(dropna=False)
    park_counts.loc[park_counts['pole_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
    park_counts.loc[park_counts['pole_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
    park_counts.loc[park_counts['pole_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
    park_counts.loc[park_counts['pole_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
    park_counts.loc[park_counts['pole_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']
#     park_counts.loc[park_counts['pole_id'] == ]
#      park_counts.loc[park_counts['pole_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
#      park_counts.loc[park_counts['pole_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
#     park_counts.loc[park_counts['pole_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
#     park_counts.loc[park_counts['pole_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
#     park_counts.loc[park_counts['pole_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']


park_counts['pole_id'].apply(set_temporal_counts)




# v_counts = park_df.loc[park_df['pole_id'] == 'N-1003']['time_slot'].value_counts(dropna=False)
# park_counts.loc[park_counts['pole_id'] == 'N-1003','am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']
# park_counts.loc[park_counts['pole_id'] == 'N-1003']



In [ ]:
max_date = dates.max()
min_date = dates.min()
days_elapsed = (max_date - min_date).days + 1 #to round off this number
days_elapsed

In [ ]:
park_counts.to_csv("Parking_Counts.csv");

## Data Visualization

## Data Analysis and Results 

## Conclusions/Discussion